In [1]:
import cv2
import numpy as np
import bisect

from typing import Tuple

# 嵌入像素差异的计算
def embending(n: int) -> Tuple[int, int, int]:
    srange = (0, 2, 4, 8, 12, 16, 24, 32, 48, 64, 96, 128, 192, 256)
    l = bisect.bisect_right(srange, n) - 1
    return srange[l], int(np.log2(srange[l + 1] - srange[l])), srange[l + 1] - 1

# 调整像素值差异
def change_diff(diff: int, l: int, r: int) -> Tuple[bool, int, int]:
    swap = False
    if l > r:
        l, r = r, l
        swap = True
    
    sg = np.sign(diff)
    ost = sg * (np.abs(diff) % 2)
    floor = sg * (np.abs(diff) // 2)

    l -= floor
    r += floor
    
    if l < 0 or r > 255 or ost > 0 and l == 0 and r == 255:
        return False, 0, 0
    
    if ost > 0 and l > (255 - r) or ost < 0 and l < (255 - r):
        l -= ost
    else:
        r += ost
    
    if swap:
        l, r = r, l

    return True, l, r

# 二进制转字节数组
def bin_to_bytes_readable(b: str) -> bytearray:
    return bytearray([int(b[i:i+8], 2) for i in range(0, len(b), 8)])

# 隐藏秘密消息
def pvd_store(img_name: str, secret: str) -> bool:
    img = cv2.imread(img_name)

    height, width = img.shape[0], img.shape[1]
    width -= width % 2
    
    data = bin(int.from_bytes(bytearray(secret.encode()), byteorder='big'))[2:]
    data = '0' * (8 - len(data) % 8) + data

    data_len = bin(len(data))[2:].zfill(32)
    data = data_len + data

    i = capacity = 0
    while i < height:
        for j in range(0, width, 2):
            for k in range(3):
                dif = max(img[i, j + 1, k], img[i, j, k]) - min(img[i, j + 1, k], img[i, j, k])

                emb, n, maxr = embending(dif)
                res, _, _ = change_diff(maxr - dif, min(img[i, j + 1, k], img[i, j, k]), max(img[i, j + 1, k], img[i, j, k]))
                if not res:
                    continue
                
                bits = data[capacity:capacity + n]
                capacity += len(bits)

                new_dif = emb + int(bits, 2)
                _, img[i, j, k], img[i, j + 1, k] = change_diff(new_dif - dif, img[i, j, k], img[i, j + 1, k])

                if capacity == len(data):
                    cv2.imwrite('./results/stego.png', img)
                    return True
        
        i += 1

    return False

# 从图像中提取隐藏的秘密消息
def pvd_unstore(img_name: str) -> str:
    img = cv2.imread(img_name)

    height, width = img.shape[0], img.shape[1]
    width -= width % 2

    capacity = -1
    result_len, is_body = '', False
    result = ''
    for i in range(height):
        for j in range(0, width, 2):
            for k in range(3):
                dif = max(img[i, j + 1, k], img[i, j, k]) - min(img[i, j + 1, k], img[i, j, k])

                emb, ln, maxr = embending(dif)
                res, _, _ = change_diff(maxr - dif, min(img[i, j + 1, k], img[i, j, k]), max(img[i, j + 1, k], img[i, j, k]))
                if not res:
                    continue
                
                secret = dif - emb

                bits = bin(secret)[2:].zfill(ln)
                if not is_body:
                    result_len += bits

                    if len(result_len) >= 32:
                        is_body = True
                        capacity = int(result_len[:32], 2)

                        if len(result_len) > 32:
                            result = result_len[32:]
                    
                    continue
                else:
                    if len(bits) + len(result) > capacity:
                        bits = bits.lstrip('0')
                        bits = bits.zfill(capacity - len(result))
                    
                    result += bits
                
                if is_body and len(result) == capacity:
                    return bin_to_bytes_readable(result).decode()
    
    return 'Error! Impossible to extract secret message!'

# 测试代码
def test_pvd():
    # 设置隐藏消息和图像路径
    img_path = './dataset/cover.jpg'  # 替换为你的图像文件路径
    secret_message = 'This is a hidden message!'
    
    # 隐藏消息到图像
    if pvd_store(img_path, secret_message):
        print('Text was successfully hidden in stego.png')
    else:
        print(f'Too long text for image {img_path}. Container "stego.png" is invalid.')
    
    # 提取隐藏的消息
    secret = pvd_unstore('./results/stego.png')
    print(f'Hidden message: {secret}')

# 调用测试
if __name__ == '__main__':
    test_pvd()


Text was successfully hidden in stego.png
Hidden message: This is a hidden message!


In [2]:
import math
from PIL import Image
from skimage.metrics import structural_similarity as ssim


def psnr(img1, img2):
    """计算两个图像之间的 PSNR"""
    mse = np.mean((np.array(img1) - np.array(img2)) ** 2)
    if mse == 0:
        return float('inf')  # 完全相同的图像
    max_pixel = 255.0
    return 20 * math.log10(max_pixel / math.sqrt(mse))

def compare_images(cover_path, stego_path):
    """对比原始图像和隐藏图像之间的差异"""
    
    cover_image = Image.open(cover_path)
    stego_image = Image.open(stego_path)

    # 计算 PSNR
    psnr_value = psnr(cover_image, stego_image)
    print(f"PSNR between cover and stego image: {psnr_value} dB")

    # 计算 SSIM
    cover_array = np.array(cover_image.convert('RGB'))
    stego_array = np.array(stego_image.convert('RGB'))
    
    ssim_value = ssim(cover_array, stego_array, multichannel=True, win_size=3)
    print(f"SSIM between cover and stego image: {ssim_value}")

# 示例用法
cover_path = "./dataset/cover.jpg"  # 载体图像路径
stego_path = "./results/stego.png"  # 隐藏图像路径

# 对比原始图像和隐藏图像
compare_images(cover_path, stego_path)


PSNR between cover and stego image: 101.17784290463221 dB
SSIM between cover and stego image: 0.9999999929573521
